In [1]:
import asyncio
from app.api.chatwoot import ChatwootHandler
import os
from dotenv import load_dotenv

load_dotenv()

# Initialize the handler
handler = ChatwootHandler(
    api_url=os.getenv('CHATWOOT_API_URL') or "",
    api_key=os.getenv('CHATWOOT_API_KEY') or "",
    account_id=os.getenv('CHATWOOT_ACCOUNT_ID') or ""
)

async def test_chatwoot():
    # Test sending a message
    conversation_id = 16  # Replace with actual conversation ID
    test_results = {}
    
    async def run_test(name: str, coro):
        try:
            result = await coro
            test_results[name] = {"status": "success", "result": result}
            print(f"✅ {name} succeeded:", result)
        except Exception as e:
            test_results[name] = {"status": "failed", "error": str(e)}
            print(f"❌ {name} failed:", e)
    
    # Test all methods
    await run_test(
        "send_message",
        handler.send_message(
            conversation_id=conversation_id,
            message="Hello, this is a test message!",
            private=False
        )
    )
    
    await run_test(
        "update_conversation_status",
        handler.update_conversation_status(
            conversation_id=conversation_id,
            status="pending"
        )
    )
    
    await run_test(
        "add_labels",
        handler.add_labels(
            conversation_id=conversation_id,
            labels=["test_label", "automated"]
        )
    )
    
    await run_test(
        "get_conversation_data",
        handler.get_conversation_data(conversation_id)
    )
    
    await run_test(
        "assign_conversation",
        handler.assign_conversation(
            conversation_id=conversation_id,
            assignee_id=1  # Replace with actual agent ID
        )
    )
    
    await run_test(
        "update_custom_attributes",
        handler.update_custom_attributes(
            conversation_id=conversation_id,
            custom_attributes={"test_key": "test_value", "automated": True}
        )
    )
    
    await run_test(
        "toggle_priority",
        handler.toggle_priority(
            conversation_id=conversation_id,
            priority="high"
        )
    )

    #run test where we set priority to garbo value not in literal
    await run_test(
        "toggle_priority",
        handler.toggle_priority(
            conversation_id=conversation_id,
            priority="garbo"
        )
    )
    
    # Print summary
    print("\n=== Test Summary ===")
    total_tests = len(test_results)
    passed_tests = sum(1 for result in test_results.values() if result["status"] == "success")
    failed_tests = total_tests - passed_tests
    
    print(f"Total tests: {total_tests}")
    print(f"Passed: {passed_tests}")
    print(f"Failed: {failed_tests}")
    
    if failed_tests > 0:
        print("\nFailed tests:")
        for name, result in test_results.items():
            if result["status"] == "failed":
                print(f"- {name}: {result['error']}")

# Run the tests
await test_chatwoot()

✅ send_message succeeded: {'id': 179, 'content': 'Hello, this is a test message!', 'inbox_id': 1, 'conversation_id': 16, 'message_type': 1, 'content_type': 'text', 'status': 'sent', 'content_attributes': {}, 'created_at': 1740577799, 'private': False, 'source_id': None, 'sender': {'id': 1, 'name': 'Agent Bot', 'avatar_url': '', 'type': 'agent_bot'}}
✅ update_conversation_status succeeded: {'meta': {'sender': {'additional_attributes': {'username': 'olin', 'language_code': 'en', 'social_telegram_user_id': '44cb3295be52ecd668b90551f4e2dfc7', 'social_telegram_user_name': 'olin'}, 'availability_status': 'offline', 'email': None, 'id': 15, 'name': 'Артём Князев', 'phone_number': None, 'identifier': None, 'thumbnail': '', 'custom_attributes': {}, 'last_activity_at': 1740572583, 'created_at': 1740572583}, 'channel': 'Channel::Telegram', 'assignee': {'id': 1, 'account_id': 1, 'availability_status': 'offline', 'auto_offline': True, 'confirmed': True, 'email': 'admin@iditelesom.org', 'available_n

Failed to toggle priority for conversation 16: Server error '500 Internal Server Error' for url 'https://chat.iditelesom.dev/api/v1/accounts/1/conversations/16'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500


❌ toggle_priority failed: Server error '500 Internal Server Error' for url 'https://chat.iditelesom.dev/api/v1/accounts/1/conversations/16'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500

=== Test Summary ===
Total tests: 7
Passed: 6
Failed: 1

Failed tests:
- toggle_priority: Server error '500 Internal Server Error' for url 'https://chat.iditelesom.dev/api/v1/accounts/1/conversations/16'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500
